In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from datetime import timedelta

subject = 'EGD-0125'
csv_file = 'clinical_data.csv'
xnat_path = 'https://bigr-rad-xnat.erasmusmc.nl'
user = ''
project = 'EGD'
## Difference in dates between sources, due to anonimization
num_days_anon = -1

res = pd.read_csv(csv_file, sep=';')
res = res.set_index('Anon_ID')
dtime = timedelta(days=num_days_anon)
with xnat.connect(xnat_path, user=user) as connection:
    xnat_project = connection.projects[project]
    for subject in res.index.values:
        dates = get_scan_dates_project(subject, xnat_project)
        dates = [d - dtime for d in dates]
        clin_dates, clin_labels = get_clinical_dates(subject)
        print(subject)
        filename = 'timelines_egd/{}.png'.format(subject)
        plot_dates(clin_dates, clin_labels, dates, filename)
    
    




In [ ]:
import xnat
def get_scan_dates(subject):
    dates = []
    with xnat.connect('https://bigr-rad-xnat.erasmusmc.nl', user='KvanGarderen') as connection:
        xnat_project = connection.projects['EGD']
        xnat_subject = xnat_project.subjects[subject]
        for exp in xnat_subject.experiments:
            date = xnat_subject.experiments[exp].date
            dates.append(date)
    return dates

def get_scan_dates_project(subject, project):
    xnat_subject = xnat_project.subjects[subject]
    dates = []
    for exp in xnat_subject.experiments:
        date = xnat_subject.experiments[exp].date
        dates.append(date)
    return dates
    

In [ ]:
import matplotlib.dates as mdates

def plot_dates(dates, labels, scan_dates, filename):
    fig, ax = plt.subplots(figsize=(10, 8), constrained_layout=True)
    ax.set(title=subject)

    levels = np.tile([-5, 5, -3, 3, -1, 1, -7, 7, -9, 9],
                     int(np.ceil(len(dates)/10)))[:len(dates)]
    markerline, stemline, baseline = ax.stem(dates, levels,
                                             linefmt="C3-", basefmt="k-",
                                             use_line_collection=True)
    
    ax.stem(scan_dates, [0]*len(scan_dates),
                                             linefmt="C3-", basefmt="k-",
                                             use_line_collection=True)

    plt.setp(markerline, mec="k", mfc="w", zorder=3)

    # annotate lines
    vert = np.array(['top', 'bottom'])[(levels > 0).astype(int)]
    for d, la, l, va in zip(dates, labels, levels, vert):
        ax.annotate(la, xy=(d, l), xytext=(-3, np.sign(l)*3),
                    textcoords="offset points", va=va, ha="right")
        ax.annotate(d.strftime('%d-%m-%Y'), xy=(d, l), xytext=(-3, np.sign(l)*15),
                    textcoords="offset points", va=va, ha="right")

    # remove y axis and spines
    ax.get_yaxis().set_visible(False)
    for spine in ["left", "top", "right"]:
        ax.spines[spine].set_visible(False)

    # format xaxis with 4 month intervals
    ax.get_xaxis().set_major_locator(mdates.YearLocator())
    ax.get_xaxis().set_major_formatter(mdates.DateFormatter("%Y"))
    plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

    ax.margins(y=0.5, x=0.2)
    plt.savefig(filename)
    plt.show()



In [ ]:
import pandas as pd
from datetime import datetime as dt
import matplotlib.dates as mdates


def get_clinical_dates(subject):
    res = pd.read_csv(csv_file, sep=';')
    res = res.set_index('Anon_ID')
    labels = ['Date_first_symptom', 
              'Date_diagn_MRI', 
              'Death_Date', 
              'Date_progression', 
              'Date_pre-ok_scan', 
              'Date_post-ok_scan',
            'Date OK1', 'Date OK2', 'Date OK3', 'Date Biopsy',
             'RT1_start', 'RT1_end', 'RT2_start', 'RT2_end',
             'Chemotherapy_start', 'Chemotherapy_end', 'Chemotherapy2_start', 'Chemotherapy2_end',
             'Chemotherapy3_start', 'Chemotherapy3_end']
    dates = []
    used_labels = []
    for l in labels:
        date = res.loc[subject, l]
        
        try:
            parsed_date = dt.strptime(date, '%m/%d/%Y')
            dates.append(parsed_date.date())
            used_labels.append(l)
        except:
            pass
        try:
            parsed_date = dt.strptime(date, '%d-%m-%Y')
            dates.append(parsed_date.date())
            used_labels.append(l)
        except:
            pass
    return dates, used_labels
    

